# How to build an advanced Chatbot with session memory using LangChain
* Advanced Chatbot LLM App.
    * Will be able to have a conversation.
    * Will remember previous interactions: will have memory.
    * Will be able to have different memories for different user sessions.
    * Will be able to remember a limited number of messages: limited memory.


## Concepts included
* Chat Model vs. LLM Model:
    *  Chat Model is based around messages.
    *  LLM Model is based around raw text.
* Chat History: allows Chat Model to remember previous interactions.

In this exercise we will use the LangChain legacy chain LLMChain. It works well, but LangChain displays a nasty deprecation warning. To avoid it, we will enter the following code:

In [1]:
import warnings
from langchain._api import LangChainDeprecationWarning

warnings.simplefilter("ignore", category=LangChainDeprecationWarning)

In [6]:
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
openai_api_key = os.environ["OPENAI_API_KEY"]

**Connect with an LLM and start a conversation with it**

* For this project, we will use OpenAI's gpt-3.5-turbo

In [7]:
from langchain_openai import ChatOpenAI

chatbot = ChatOpenAI(model="gpt-3.5-turbo")

* Human Message: the user input.

In [8]:
from langchain_core.messages import HumanMessage

messagesToTheChatbot = [
    HumanMessage(content="My favourite color is blue."),
]

**Call the ChatModel (the LLM)**

In [9]:
chatbot.invoke(messagesToTheChatbot)

AIMessage(content="Blue is such a calming and serene color. It's often associated with peace, tranquility, and stability. What do you like most about the color blue?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 32, 'prompt_tokens': 13, 'total_tokens': 45}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f920f1d8-81e1-4ff3-aae8-3ab62c69cd57-0', usage_metadata={'input_tokens': 13, 'output_tokens': 32, 'total_tokens': 45})

## Check if the Chatbot remembers your favorite color.

In [10]:
chatbot.invoke([
    HumanMessage(content="What is my favorite color?")
])

AIMessage(content="I'm sorry, I do not have that information. Can you please tell me what your favorite color is?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 13, 'total_tokens': 35}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-ec5b6f97-d537-4afe-8f99-c00cf916b44d-0', usage_metadata={'input_tokens': 13, 'output_tokens': 22, 'total_tokens': 35})

* As you can see, our Chatbot cannot remember our previous interaction.

## Let's add memory to our Chatbot
* We will use the ChatMessageHistory package.
* We will save the Chatbot memory in a python dictionary called chatbotMemory.
* We will define the get_session_history function to create a session_id for each conversation.
* We will use the built-in runnable RunnableWithMessageHistory.

**RunnableWithMessageHistory** : Associate the get_session_history function with the original chatbot that was created.

In [11]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

chatbotMemory = {}

# input: session_id, output: chatbotMemory[session_id]
def get_session_history(session_id: str) -> BaseChatMessageHistory:
    if session_id not in chatbotMemory:
        chatbotMemory[session_id] = ChatMessageHistory()
    return chatbotMemory[session_id]


chatbot_with_message_history = RunnableWithMessageHistory(
    chatbot, 
    get_session_history
)

**BaseChatMessageHistory**
Is what is called an **abstract base class** in Python. 

Here's a simpler breakdown of what it means for `BaseChatMessageHistory` to be an abstract base class:

#### Let's explain the previous code in simple terms
The previous code manages the chatbot's memory of conversations based on session identifiers. Here’s a breakdown of what the different components do:

1. **chatbotMemory**:
    - `chatbotMemory = {}`: This initializes an empty dictionary where session IDs and their respective chat histories will be stored.

2. **get_session_history Function**:
    - This function, `get_session_history`, takes a `session_id` as an argument and returns the chat history associated with that session.
    - If a chat history for the given `session_id` does not exist in `chatbotMemory`, a new instance of `ChatMessageHistory` is created and assigned to that `session_id` in the dictionary.
    - The function ensures that each session has its own unique chat history, stored and retrieved using the session ID.

3. **chatbot_with_message_history**:
    - `chatbot_with_message_history = RunnableWithMessageHistory(chatbot, get_session_history)`: This line creates an instance of `RunnableWithMessageHistory` using two arguments: `chatbot` and `get_session_history`.
    - The `chatbot` is passed along with the `get_session_history` function. This setup integrates the chatbot with the functionality to handle session-specific chat histories, allowing the chatbot to maintain continuity and context in conversations across different sessions.
    - **Learn more about RunnableWithMessageHistory** [here](https://python.langchain.com/v0.1/docs/expression_language/how_to/message_history/).

Overall, the code organizes and manages a chatbot's memory, enabling it to handle different sessions with users effectively by remembering previous messages within each session.

#### RunnableWithMessageHistory
**When invoking a new RunnableWithMessageHistory, we specify the corresponding chat history using a configurable parameter**. Let's say we want to create a chat memory for one user session, let's call it session1:

In [19]:
session1 = {"configurable": {"session_id": "001"}}

In [20]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite color is black.")],
    config=session1,
)

responseFromChatbot.content

"Thank you for clarifying! Black is a classic and sophisticated choice that exudes elegance and mystery. It's a versatile color that can be easily incorporated into any style or aesthetic. Do you find that black complements your personality or preferences in some way?"

In [26]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Based on your responses, it seems like your favorite color is black.'

**Let's now change the session_id and see what happens**

Now let's create a chat memory for another user session, let's call it session2:

In [23]:
session2 = {"configurable": {"session_id": "002"}}

If the chatbot is using this new memory for session2, it will not be able to remember anything from the previous conversation in the session1:

In [25]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session2,
)

responseFromChatbot.content

"I'm sorry, I cannot determine your favorite color without you telling me. What is your favorite color?"

## Let's go back to session1 and see if the memory is still there

In [27]:
session1 = {"configurable": {"session_id": "001"}}

In [28]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What's my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'It seems like your favorite color is black.'

As we can see, the chatbot is now able to remember the session1 conversation.

**Our ChatBot has session memory now. Let's check if it remembers the conversation from session2.**

In [29]:
session2 = {"configurable": {"session_id": "002"}}

In [30]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="Mi name is Marcela.")],
    config=session2,
)

responseFromChatbot.content

'Nice to meet you, Marcela! Do you have a favorite color that you would like to share with me?'

In [31]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my name?")],
    config=session2,
)

responseFromChatbot.content

'Your name is Marcela.'

In [32]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="What is my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Based on your previous responses, it seems like your favorite color is black.'

**Our chatBot now remembers each of our conversations.**

**The importance to manage the Conversation History**
* The memory of a chatbot is included in the context window of the LLM so, if left unmanaged, can potentially overflow it.
* **We are now going to learn how to limit the size of the memory of a chatbot**.
* First, let's take a look at what is in the memory of our chatbot:

In [33]:
print(chatbotMemory)

{'001': InMemoryChatMessageHistory(messages=[HumanMessage(content='My favorite color is red.'), AIMessage(content="That's a bold and vibrant choice! Red is often associated with passion, energy, and determination. It's a strong and eye-catching color that can make a statement in any setting. What do you love most about the color red?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 13, 'total_tokens': 60}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b652f059-222c-4081-97ea-c97c80702073-0', usage_metadata={'input_tokens': 13, 'output_tokens': 47, 'total_tokens': 60}), HumanMessage(content='My favorite color is black.'), AIMessage(content='Black is a timeless and versatile color choice. It is often associated with elegance, sophistication, and mystery. Black can be both bold and understated, making it a popular choice for fashion, design, and decor. W

* Now, **let's define a function to limit the number of messages stored in memory and add it to our chain with .assign**.

In [34]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnablePassthrough


def limited_memory_of_messages(messages, number_of_messages_to_keep=2):
    return messages[-number_of_messages_to_keep:]

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

limitedMemoryChain = (
    RunnablePassthrough.assign(messages=lambda x: limited_memory_of_messages(x["messages"]))
    | prompt 
    | chatbot
)

* The limited_memory_of_messages function allows you to trim the list of stored messages, keeping only a specified number of the latest ones. For example, if you have a long list of messages and you only want to keep the last two, this function will do that for you.
* The lambda function works in conjunction with the `limited_memory_of_messages` function. Here’s a simple breakdown:

    1. **Lambda Function**: The `lambda` keyword is used to create a small anonymous function in Python. The `lambda` function defined here takes one argument, `x`.

    2. **Function Argument**: The argument `x` is expected to be a dictionary that contains a key named `"messages"`. The value associated with this key is a list of messages.

    3. **Function Body**: The body of the `lambda` function calls the `limited_memory_of_messages` function. It passes the list of messages found in `x["messages"]` to this function.

    4. **Default Behavior of limited_memory_of_messages**: Since the `lambda` function does not specify the `number_of_messages_to_keep` parameter when it calls `limited_memory_of_messages`, the latter will default to keeping the last 2 messages from the list (as defined by the earlier function).

In essence, the `lambda` function is a shorthand way to apply the `limited_memory_of_messages` function to the message list contained within a dictionary. It automatically trims the list to the last two messages.

**Let's now create our new chatbot with limited message history**:

In [35]:
chatbot_with_limited_message_history = RunnableWithMessageHistory(
    limitedMemoryChain,
    get_session_history,
    input_messages_key="messages",
)

**Let's add 2 more messages to the session1 conversation:**

In [36]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite vehicles are Vespa scooters.")],
    config=session1,
)

responseFromChatbot.content

'Vespa scooters are a classic and iconic choice for transportation! With their retro design and compact size, Vespa scooters are popular for their style, convenience, and fun factor. Do you own a Vespa scooter, or do you just admire them from afar?'

In [37]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="My favorite city is San Francisco.")],
    config=session1,
)

responseFromChatbot.content

'San Francisco is a vibrant and diverse city known for its iconic landmarks, stunning views, and unique culture. From the Golden Gate Bridge to the historic cable cars, San Francisco offers a mix of history, art, and innovation. What do you love most about San Francisco?'

**The chatbot memory has now 4 messages. Let's check the Chatbot with limited memory.**
* Remember, this chatbot only remembers the last 2 messages, so if we ask her about the first message she should not remember it.

In [38]:
responseFromChatbot = chatbot_with_limited_message_history.invoke(
    {
        "messages": [HumanMessage(content="what is my favorite color?")],
    },
    config=session1,
)

responseFromChatbot.content

"I'm sorry, but as an AI assistant, I don't have access to personal information about you such as your favorite color. Can I help you with anything else?"

* The chatbot with limited memory has behaved as we expected.

## Finally, let's compare the previous response with the one provided by the Chatbot with unlimited memory

In [39]:
responseFromChatbot = chatbot_with_message_history.invoke(
    [HumanMessage(content="what is my favorite color?")],
    config=session1,
)

responseFromChatbot.content

'Based on the information you have provided, it seems like your favorite color is black.'

* As you can see, this chatbot remembers our first message.

In [40]:
print(chatbotMemory)

{'001': InMemoryChatMessageHistory(messages=[HumanMessage(content='My favorite color is red.'), AIMessage(content="That's a bold and vibrant choice! Red is often associated with passion, energy, and determination. It's a strong and eye-catching color that can make a statement in any setting. What do you love most about the color red?", additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 47, 'prompt_tokens': 13, 'total_tokens': 60}, 'model_name': 'gpt-3.5-turbo-0125', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-b652f059-222c-4081-97ea-c97c80702073-0', usage_metadata={'input_tokens': 13, 'output_tokens': 47, 'total_tokens': 60}), HumanMessage(content='My favorite color is black.'), AIMessage(content='Black is a timeless and versatile color choice. It is often associated with elegance, sophistication, and mystery. Black can be both bold and understated, making it a popular choice for fashion, design, and decor. W